# Question answering system

In this small project, I implement a question answering system (QAS) with a dual encoder architecture trained on the Ubuntu Dialogue Corpus.

### First, define the problem

There are many types of QAS and they are all different to each other in some way. Some QASs answer questions according to a text corpus in a way similar to a reading comprehension task, some are able to reply to open-ended questions in scenarios such as customer service or conversation. Also, there are different ways for QAS to produce responeses: it can be generated from a generative model, or it can be selected from a predefined set using a retrieval model.

After some search, I decide to go with the Ubuntu Dialogue Corpus and build a retrieval model using dual encoder for the following reasons:

  * It is more similar to a chatbot or a dialogue system compare to other tasks and datasets.
  * There are many literatures and research papers that talk about or use this dataset.
  * I find a [preprocessed version](https://drive.google.com/file/d/1VjVzY0MqKj0b-q_KXnaHC49qCw3iDqEY/view) of this dataset so it saves my time.
  * A retrieval model is easier to train comparing to a generative one.

### Load the preprocessed dataset, dictionary, and a pretrained word embedding

I downloade the preprocessed dataset from [here](https://drive.google.com/file/d/1VjVzY0MqKj0b-q_KXnaHC49qCw3iDqEY/view). Following the [original paper](https://arxiv.org/abs/1506.08909) of the dataset, I use GloVe as the pretrained word embedding, which can be downloaded from [here](https://nlp.stanford.edu/projects/glove/).

In [1]:
import numpy as np

from utils import load_data, load_dictionary, dataset_to_corpus
from consts import data_dir, model_dir

Using TensorFlow backend.


In [2]:
# Load the splitted data (only use a small subset)
train_c, train_r, train_l, \
dev_c, dev_r, dev_l, \
test_c, test_r, test_l = load_data(ratio=0.05)

Loading data with sampling ratio 0.05
# training data: 50000
# validation data: 9780
# testing data: 9460


The preprocessed dataset is already tokenized and looks like this.

In [3]:
train_c[0, :]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1766,    3, 1192,  923,    1,   91,   10,    7,   13,    6,    5,
         38, 1685,   91,  113,   17,    7,  561,   

To know the original words, we need to load also the dictionary.

In [4]:
# Load the dictionary
word_index, inv_word_index, MAX_SEQUENCE_LENGTH = load_dictionary()

print('Context:\n', dataset_to_corpus([train_c[0, :]], inv_word_index)[0], '\n')
print('Response:\n', dataset_to_corpus([train_r[0, :]], inv_word_index)[0], '\n')

Context:
 god i hate computers eou why do it have to be so complicate why ca n't it ever be simple eou eot what be your problem eou eot 

Response:
 ubuntu be mess up real bad and i 'm be tell i should backup my hd before i try to fix it eou 



`eou` and `eot` are specials token that stand for "end of utterance" and "end of turn", respectively.

Because I want to build a retrieval model, I need both "true response" and "false response" in the dataset to train the model using supervised learning. Here is an example of a false response:

In [5]:
data_id = 4
print('True or false response:', train_l[data_id], '\n')
print('Context:\n', dataset_to_corpus([train_c[data_id, :]], inv_word_index)[0], '\n')
print('Response:\n', dataset_to_corpus([train_r[data_id, :]], inv_word_index)[0], '\n')

True or false response: 0 

Context:
 as you like eou ya get a brand new burner eou becauase your browser doesnt know its sposed to dislay php eou eot well i have add a couple of line to my apache2 conf addtype application x httpd php php eou also a line for phps eou eot i dont know how to make the browser render php eou both eou what libs eou eot http paste ubuntulinux nl 4009 eou eot it be a bad idea to put anything in usr lib eou use usr local lib instead eou eot someone else inform me after i copy p eou but thank p eou eot there be more to it than that eou eot yar i be jump the gun eou will remove var cache debconf config dat do any damage dpkg reconfigure be complain about a lock issue eou eot ah that 's why eou where do you get the game eou eot www racer nl eou eot 

Response:
 i have instal them but not sure they get use also it dont include tahoma eou 



### Build and train the model

The model I build is a LSTM dual encoder, i.e. a siamese-like architecture. I experimente with several variations of the model, such as the output dimension of the encoder and the way to compute the prediction from the encoder's outputs.

In the original Ubuntu dataset paper, the last layer of the network is $\sigma(cMr^T+b)$, where $M, b$ are trainable variables and $c, r$ are the output of the encoder of the given context and response, respectively. It is argued in the paper that the transformation $M$ can be seen as a way to generate a response $r'=cM$ and the rest of the formula measures the similarity between $r$ and $r'$. I also try to measure the similarity between $c$ and $r$ directly, i.e. setting $M=I$ and $b=0$.

Due to the limited computation power, an normal Intel CPU on a laptop without powerful GPU, I train the model with only a small portion of the dataset, which may affect the performance of neural networks.

Please refer to `train.py` for the code. To retrain the model, use the command `$ python3 train.py`.

### Evaluate the model

Following the original paper of the dataset, I use TF-IDF as a baseline method and evaluate the results using recall@k with different number of false responses. The test set is constructed such that for each context there is one true response and other nine randomly selected false responses. The model ranks the responses based on its output and recall@k computes the number of times the true response appears in the top-k list divided by the total number of test data.

In [6]:
import pickle
from utils import recall_at_k
from tfidf import tfidf

In [7]:
# List of models and their predictions
# The prediction files are generated using the script predict.py
pred_filenames = {
    'DE-no transform (100)': 'dot_0.05/model.2.0.5783862575187761.pred.pkl',
    'DE (100)': 'mat_0.05/model.2.0.6053460450503968.pred.pkl',
    'DE (50)': 'mat_0.05_dim_50/model.2.0.6308096857158685.pred.pkl',
}

In [8]:
# Evaluate the models
for model_name, pred_filename in pred_filenames.items():
    with open(str(model_dir / pred_filename), 'rb') as f:
        y_pred = pickle.load(f)

    print('Model:', model_name)
    for group_size in [2, 10]:
        for k in [1, 2, 5]:
            if k >= group_size:
                break
            r = recall_at_k(y_pred, k, group_size)
            print('recall@{} (out of {:2d}): {}'.format(k, group_size, r))
    print('')

Model: DE-no transform (100)
recall@1 (out of  2): 0.7769556025369979
recall@1 (out of 10): 0.3752642706131078
recall@2 (out of 10): 0.5750528541226215
recall@5 (out of 10): 0.8266384778012685

Model: DE (100)
recall@1 (out of  2): 0.6976744186046512
recall@1 (out of 10): 0.27167019027484146
recall@2 (out of 10): 0.4281183932346723
recall@5 (out of 10): 0.7526427061310782

Model: DE (50)
recall@1 (out of  2): 0.7135306553911205
recall@1 (out of 10): 0.2727272727272727
recall@2 (out of 10): 0.44608879492600423
recall@5 (out of 10): 0.7706131078224101



In [9]:
# The result of TF-IDF
tfidf()

Loading data ...
Loading data with sampling ratio 0.05
# training data: 50000
# validation data: 9780
# testing data: 9460
Transforming to text corpuses ...
Fitting a tfidf model ...
Transforming to tfidf features ...
Predicting ...
Results:

recall@1 (1 options): 0.7632135306553911
recall@1 (9 options): 0.514799154334038
recall@2 (9 options): 0.6088794926004228
recall@5 (9 options): 0.7727272727272727


| . | TF-IDF | DE (100) | DE (50) | DE-no transform (100) |
|--|--|--|--|--|
| recall@1 (out of 2) | 0.763 | 0.698 | 0.714 | 0.777 |
| recall@1 (out of 10) | 0.515 | 0.272 | 0.273 | 0.375 |
| recall@2 (out of 10) | 0.609 | 0.428 | 0.446 | 0.575 |
| recall@5 (out of 10) | 0.773 | 0.753 | 0.771 | 0.827 |

This is the result of the experiment. The three models on the right are dual encoders and the last one has $M=I, b=0$ set in the last layer as described in the previous section. The number in the parentheses is the output dimension of the encoders.

We see that all the numbers are a lot greater than a random model would produce, showing that the models are working properly. However, TF-IDF still performes better in most cases. The reason for this is very likely to be the subsampled dataset mentioned in the previous section. In this experiment, I subsample only 5% of the dataset, which reduces the size of the training set from 1,000,000 to 50,000, so that the trainings finish in a reasonable amount of time. But such little data may not be enough for the huge network: a huge word embedding layer with size about 400,000 * 300 and a matrix $M$ of size 100 * 100 at the end. This can be confirmed by comparing between last three columns. Indeed, we see that the performance is improved in all metrics if we reduce the output dimension of the encoder or set $M$ to identity, which both reduce the number of parameters in the model. Reducing the dimension of word embedding might help, but training a new word embedding also needs a lot of data.

Regarding the evaluation metric, while it makes sense to use recall@k in this case, it would also be interesting to try some other metrics mentioned in this [paper](https://arxiv.org/abs/1603.08023), such as greedy matching, embedding average and vector extrema, in the future.